In [ ]:
%load_ext autoreload
%autoreload 2

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt

plt.rcParams["figure.figsize"] = (10, 7)
plt.rcParams["font.size"] = 12

In [ ]:
import inspect

import numpy as np
import xarray as xr
import xarray_sentinel

from sarsen import apps, geocoding, orbit, scene

# uncomment to check that the code below is in sync with the implementation
# print(inspect.getsource(apps.terrain_correction))

# define input and load data

In [ ]:
product_urlpath = (
    "data/S1B_IW_SLC__1SDV_20211223T051121_20211223T051148_030148_039993_BA4B.SAFE/"
)
measurement_group = "IW3/VV"
dem_urlpath = "data/Rome-10m-DEM.tif"
orbit_group = None
calibration_group = None
output_urlpath = "Rome-10m-GTC-SLC.tif"
correct_radiometry = False
interp_method = "nearest"
multilook = None
grouping_area_factor = (1.0, 1.0)
open_dem_raster_kwargs = {"chunks": {}}
kwargs = {"chunks": {"pixel": 2048}}

In [ ]:
!ls -d {product_urlpath}
!ls -d {dem_urlpath}

In [ ]:
orbit_group = orbit_group or f"{measurement_group}/orbit"
calibration_group = calibration_group or f"{measurement_group}/calibration"

measurement_ds = xr.open_dataset(product_urlpath, engine="sentinel-1", group=measurement_group, **kwargs)  # type: ignore
measurement = measurement_ds.measurement

dem_raster = scene.open_dem_raster(dem_urlpath, **open_dem_raster_kwargs)

orbit_ecef = xr.open_dataset(product_urlpath, engine="sentinel-1", group=orbit_group, **kwargs)  # type: ignore
position_ecef = orbit_ecef.position
calibration = xr.open_dataset(product_urlpath, engine="sentinel-1", group=calibration_group, **kwargs)  # type: ignore
beta_nought_lut = calibration.betaNought

# scene

In [ ]:
dem_raster

In [ ]:
_ = dem_raster.plot()

In [ ]:
%%time
dem_ecef = scene.convert_to_dem_ecef(dem_raster)
dem_ecef

# acquisition

In [ ]:
measurement

In [ ]:
%%time
acquisition = apps.simulate_acquisition(position_ecef, dem_ecef)
acquisition

In [ ]:
%%time
beta_nought = xarray_sentinel.calibrate_intensity(measurement, beta_nought_lut)
beta_nought

In [ ]:
%%time
coordinate_conversion = None
if measurement_ds.attrs["sar:product_type"] == "GRD":
    coordinate_conversion = xr.open_dataset(
        product_urlpath,
        engine="sentinel-1",
        group=f"{measurement_group}/coordinate_conversion",
        **kwargs,
    )  # type: ignore
    ground_range = xarray_sentinel.slant_range_time_to_ground_range(
        acquisition.azimuth_time,
        acquisition.slant_range_time,
        coordinate_conversion,
    )
    interp_kwargs = {"ground_range": ground_range}
elif measurement_ds.attrs["sar:product_type"] == "SLC":
    interp_kwargs = {"slant_range_time": acquisition.slant_range_time}
    if measurement_ds.attrs["sar:instrument_mode"] == "IW":
        beta_nought = xarray_sentinel.mosaic_slc_iw(beta_nought)
else:
    raise ValueError(
        f"unsupported sar:product_type {measurement_ds.attrs['sar:product_type']}"
    )

In [ ]:
%%time
geocoded = apps.interpolate_measurement(
    beta_nought,
    multilook=multilook,
    azimuth_time=acquisition.azimuth_time,
    interp_method=interp_method,
    **interp_kwargs,
)

geocoded

In [ ]:
geocoded.rio.set_crs(dem_raster.rio.crs)
geocoded.rio.to_raster(
    output_urlpath,
    dtype=np.float32,
    tiled=True,
    blockxsize=512,
    blockysize=512,
    compress="ZSTD",
    num_threads="ALL_CPUS",
)

In [ ]:
_ = geocoded.plot(vmax=1.0)